# 11：ハイパーパラメータの探索と検証データ
---
## 目的
多層パーセプトロン (Multi Layer Perceptoron; MLP) を用いたMNISTデータセット文字認識を通じて，ハイパーパラメータの探索・検証および検証データの役割について理解する．

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gzip

## データセットのダウンロードと読み込み

まずはじめに，`wget`コマンドを使用して，MNISTデータセットをダウンロードします．

In [ ]:
!wget -q http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz -O train-images-idx3-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz -O train-labels-idx1-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz -O t10k-images-idx3-ubyte.gz
!wget -q http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz -O t10k-labels-idx1-ubyte.gz

次に，ダウンロードしたファイルからデータを読み込みます．詳細は前回までのプログラムを確認してください．

In [ ]:
# load images
with gzip.open('train-images-idx3-ubyte.gz', 'rb') as f:
    x_train = np.frombuffer(f.read(), np.uint8, offset=16)
x_train = x_train.reshape(-1, 784)

with gzip.open('t10k-images-idx3-ubyte.gz', 'rb') as f:
    x_test = np.frombuffer(f.read(), np.uint8, offset=16)
x_test = x_test.reshape(-1, 784)

with gzip.open('train-labels-idx1-ubyte.gz', 'rb') as f:
    y_train = np.frombuffer(f.read(), np.uint8, offset=8)

with gzip.open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
    y_test = np.frombuffer(f.read(), np.uint8, offset=8)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

## 検証データの作成

ネットワークの学習やモデルの定義には多くのハイパーパラメータが存在します．例えば，
* ネットワークのハイパーパラメータ
  * 中間層の層数
  * 中間層のユニット数
  * Dropoutを適用するかどうか（Dropoutを用いる場所やdropout ratio）
  * Batch Normalizationを適用するかどうか

* 学習のハイパーパラメータ
  * 学習率
  * 学習回数
  * ミニバッチサイズ

などが挙げられます．

最適なハイパーパラメータを決定するために使用するデータを**検証データ（validation data）**と呼びます．

MNISTデータセットには学習データ（train data）とテストデータ（test data）しか存在しません．このように，専用の検証データが存在しないデータセットや個人が作成したオリジナルのデータセットでは，学習データの一部を検証データとして使用します．

以下では，MNISTデータセットの学習データを分割し，学習および検証データを作成します．
まず，検証データに使用するデータの割合を`validation_ratio`として定義します．
今回は学習データの20%を検証データとして使用することとし，0.2と指定します．
この割合に基づいて，学習データの20%となるサンプル数を`n_val`として計算します．

その後，学習および検証データになるよう，データを分割します．

実行すると，60000枚の学習サンプルの20%である12000枚が検証データ，残りの80%の48000枚が学習データになるよう分割されていることがわかります．


In [ ]:
validation_ratio = 0.2   # 検証に使用するデータの割合
n_train_original = x_train.shape[0]
n_val = int(n_train_original * validation_ratio)

# 検証データ
x_val = x_train[0:n_val]
y_val = y_train[0:n_val]

# 学習データ
x_train = x_train[n_val:]
y_train = y_train[n_val:]

print("train      :", x_train.shape, y_train.shape)
print("validation :", x_val.shape, y_val.shape)
print("test       :", x_test.shape, y_test.shape)

## ネットワークモデルの定義
次に，ニューラルネットワーク（多層パーセプトロン）を定義します．

まずはじめに，ネットワークの定義に必要な関数を定義します．

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    grad = np.zeros(x.shape)
    grad[x > 0] = 1
    return grad

def softmax(x):
    if x.ndim == 2:
        x = x.T
        # x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    # x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

次に，上で定義した関数を用いてネットワークを定義します．
ここでは，入力層，中間層，出力層から構成される多層パーセプトロンとします．ネットワーク定義についての詳細は割愛しますので，前回までの資料を参照してください．

In [ ]:
class MLP:

    def __init__(self, input_size, hidden_size, output_size, act_func='sigmoid', w_std=0.01):
        self.w1 = w_std * np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)
        self.w2 = w_std * np.random.randn(hidden_size, hidden_size)
        self.b2 = np.zeros(hidden_size)
        self.w3 = w_std * np.random.randn(hidden_size, output_size)
        self.b3 = np.zeros(output_size)

        # 使用する活性化関数の選択
        if act_func == 'sigmoid':
            self.act = sigmoid
            self.act_grad = sigmoid_grad
        elif act_func == 'relu':
            self.act = relu
            self.act_grad = relu_grad
        else:
            print("ERROR")

        self.grads = {}

    def forward(self, x):
        self.h1 = np.dot(x, self.w1) + self.b1
        self.h2 = self.act(self.h1)
        self.h3 = np.dot(self.h2, self.w2) + self.b2
        self.h4 = self.act(self.h3)
        self.h5 = np.dot(self.h4, self.w3) + self.b3
        self.y = softmax(self.h5)
        return self.y

    def backward(self, x, t):
        batch_size = x.shape[0]
        self.grads = {}
        
        t = np.identity(10)[t]
        dy = (self.y - t) / batch_size

        self.grads['w3'] = np.dot(self.h4.T, dy)
        self.grads['b3'] = np.sum(dy, axis=0)

        d_h4 = np.dot(dy, self.w3.T)
        d_h3 = self.act_grad(self.h3) * d_h4
        self.grads['w2'] = np.dot(self.h2.T, d_h3)
        self.grads['b2'] = np.sum(d_h3, axis=0)
        
        d_h2 = np.dot(d_h3, self.w2.T)
        d_h1 = self.act_grad(self.h1) * d_h2
        self.grads['w1'] = np.dot(x.T, d_h1)
        self.grads['b1'] = np.sum(d_h1, axis=0)
        
    def update_parameters(self, lr=0.1):
        self.w1 -= lr * self.grads['w1']
        self.b1 -= lr * self.grads['b1']
        self.w2 -= lr * self.grads['w2']
        self.b2 -= lr * self.grads['b2']  
        self.w3 -= lr * self.grads['w3']
        self.b3 -= lr * self.grads['b3']  

## 学習およびハイパーパラメータ探索の準備




### 探索時に共通するパラメータの設定

まずはじめに，探索時に共通するパラメータを定義します．
具体的には，入力層のユニット数`input_size`および出力層のユニット数`output_size`は，探索時には共通であるため，事前に定義しておきます．
また，学習，検証データのサンプル数もすでに決まっているため，変数として定義しておきます．

また，精度および誤差を算出するための関数（`multiclass_classification_accuracy`, `cross_entropy`）も共通して使用するため，ここで事前に定義を行います．

In [ ]:
input_size = x_train.shape[1]
output_size = 10

num_train_data = x_train.shape[0]
num_val_data = x_val.shape[0]
num_test_data = x_test.shape[0]

# 学習途中の精度を確認するための関数
def multiclass_classification_accuracy(pred, true):
    clf_res = np.argmax(pred, axis=1)
    return np.sum(clf_res == true).astype(np.float32)

# 学習中の誤差を確認するための関数
def cross_entropy(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### 探索するパラメータの設定

次に，検証データを用いて最適な値を求めたいパラメータを定義します．

ここでは，中間層のユニット数の最適な値を`[32, 64, 128, 256]`の中から求めるものとし，他のパラメータは固定します．

また，パラメータの探索には複数回の学習を行う必要があり計算に多くの時間を要します．
ここでは，実習中の計算時間削減のために，学習エポック数を20と小さい値に設定します．




In [ ]:
hidden_size = 64
hidden_list = [32, 64, 128, 256]

batch_size = 100

learning_rate = 0.01
# learning_rate_list = [0.1, 0.05, 0.01, 0.005, 0.001]

epoch_num = 20

learning_rate = 0.01
activation_function = 'relu'

## ハイパーパラメータ探索

ハイパーパラメータ探索を行います．

まず，各パラメータでの結果を保存するためのリスト`param_search_list`を作成します．

次に，for文を用いて探索したいハイパーパラメータを一つづつ指定し，ネットワークの学習と検証データでの精度を求めます．
ネットワークの学習プログラムは前回までのもの同様のため詳細は割愛します．

学習が終了すると，探索したパラメータの値やその時の誤差および精度の推移のデータを辞書型オブジェクト`result`に格納し，それを`param_search_list`に保存します．

これを繰り返すことで，各パラメータの値を用いた場合の精度や学習推移を確認比較することが可能となり，より精度の高いネットワークを構築するためのあたりをつけることができます．

In [ ]:
param_search_list = []

for hidden_size in hidden_list:
    model = MLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, act_func=activation_function)

    epoch_list = []
    train_loss_list = []
    train_accuracy_list = []
    val_accuracy_list = []

    iteration = 0
    for epoch in range(1, epoch_num + 1):
        sum_accuracy, sum_loss = 0.0, 0.0
        
        perm = np.random.permutation(num_train_data)
        for i in range(0, num_train_data, batch_size):
            x_batch = x_train[perm[i:i+batch_size]]
            y_batch = y_train[perm[i:i+batch_size]]
            
            y = model.forward(x_batch)
            sum_accuracy += multiclass_classification_accuracy(y, y_batch)
            sum_loss += cross_entropy(y, y_batch)
            
            model.backward(x_batch, y_batch)
            model.update_parameters(lr=learning_rate)

            iteration += 1
        
        # 検証データでの精度の確認
        val_correct_count = 0
        for i in range(num_val_data):
            input = x_val[i:i+1]
            label = y_val[i:i+1]
            y = model.forward(input)
            
            val_correct_count += multiclass_classification_accuracy(y, label)

        # 学習途中のlossと精度の保存
        epoch_list.append(epoch)
        train_loss_list.append(sum_loss / num_train_data)
        train_accuracy_list.append(sum_accuracy / num_train_data)
        val_accuracy_list.append(val_correct_count / num_val_data)

        # print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch,
        #                                                           sum_loss / num_train_data,
        #                                                           sum_accuracy / num_train_data))

    # 探索した結果の保存
    result = {'lr': learning_rate, 'n_hidden': hidden_size,
              'val_acc': val_accuracy_list,
              'train_acc': train_accuracy_list,
              'train_loss': train_loss_list}
    param_search_list.append(result)
    print(result['lr'], result['n_hidden'], result['train_acc'][-1], result['val_acc'][-1])

## 探索結果の確認

`param_search_list`に保存しておいた各ハイパーパラメータに対する結果を確認します．

まず，各結果の数値をprintします．
その後，各パラメータでの学習推移（検証データ）をプロットし比較します．

In [ ]:
# 学習結果の表示
for ps in param_search_list:
    print("lr:", ps['lr'],
          "hidden size", ps['n_hidden'],
          "train accuracy:", ps['train_acc'][-1],
          "validation accuracy:", ps['val_acc'][-1])

# グラフプロット
plt.figure()
for ps in param_search_list:
    plt.plot(ps['val_acc'], label='hidden=%d' % ps['n_hidden'])
plt.xlabel("epoch")     # x軸ラベル
plt.ylabel("accuracy")  # y軸ラベル
plt.legend()            # 凡例
plt.show()

## 課題
1. 他のパラメータについても探索をして最適な値を見つけよう
2. 複数種類のパラメータの最適な組み合わせを求めよう